In [ ]:
import os
import pandas as pd

import utils

In [ ]:
git_root_path = utils.get_git_root(os.getcwd())
data_path = os.path.join(git_root_path, utils.DATA_DIR_NAME)

training_csv_path = os.path.join(data_path, utils.TRAINING_FILENAME + "." + utils.INPUT_DATA_FORMAT)
submit_csv_path = os.path.join(data_path, utils.SUBMIT_FILENAME + "." + utils.INPUT_DATA_FORMAT)
submission_format_csv_path = os.path.join(data_path, utils.SUBMISSION_FORMAT_FILENAME + "." + utils.INPUT_DATA_FORMAT)
metadata_csv_path = os.path.join(data_path, utils.METADATA_FILENAME + "." + utils.INPUT_DATA_FORMAT)


In [ ]:
training_df = pd.read_csv(training_csv_path, sep = ";")
submission_df = pd.read_csv(submit_csv_path, sep=";")
metadata_df = pd.read_csv(metadata_csv_path, sep=";", index_col = 0).sort_index()
submission_format_df = pd.read_csv(submission_format_csv_path, sep=";", index_col = 0).sort_index()

In [ ]:
metadata_df.head()

In [ ]:
submission_format_df.sort_index()

In [ ]:
#training_df = training_df.set_index(["timestamp", "period_id", "site_id"])
per_1_training_df = training_df.iloc[training_df.index.get_level_values('period_id') == 1]
per_1_training_df = per_1_training_df.sort_index(level="site_id")
per_1_training_df_subset = per_1_training_df[["actual_consumption", "actual_pv", "price_buy_00", "price_sell_00"]]
#grid_energy_without_battery =  #* (price_without_battery / 1000.)
per_1_training_df_subset["grid_energy_without_battery"] = (per_1_training_df_subset["actual_consumption"] - per_1_training_df_subset["actual_pv"]).astype(float)

def get_money_spent_without_batt(row):
    
    print(row)
    price =  float(row["price_buy_00"]) \
        if float(row["grid_energy_without_battery"] > 0) \
        else float(row["price_sell_00"])
    
    return ((price / 1000.) * float(row["grid_energy_without_battery"]))
        

per_1_training_df_subset["money_spent_without_battery"] = per_1_training_df_subset.apply(lambda x: get_money_spent_without_batt(x))
per_1_training_df_subset.head()

In [ ]:
per_1_training_df_subset.to_csv(os.path.join(data_path, "exploration", "period_1_training.csv"))


In [ ]:
submission_df[["timestamp", "period_id", "site_id", "actual_consumption", "load_00", "actual_pv", "pv_00", "price_buy_01", "price_sell_01"]].head()

In [ ]:
sorted(training_df["site_id"].unique())

In [ ]:
metadata_df = pd.read_csv(metadata_csv_path, sep = ";")

In [ ]:
metadata_df

In [ ]:
site_ids = sorted(submission_df["site_id"].unique())